In [58]:
import pandas as pd 
import csv
import requests
import numpy as np
import matplotlib.pyplot as plt

import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    matthews_corrcoef
)
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

#from autogluon.tabular import TabularPredictor

import time
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from scipy.stats import uniform, poisson

In [69]:
test = pd.read_csv('test_data.csv')
train = pd.read_csv('train_data.csv')

#on drop le label et les noms car pas de liens ( à mon avis)
y = train.Survived
feat = ['Name', 'Survived']
X_train = train.drop(columns = feat)
X_test = test.drop(columns = 'Name')

#X_train, X_test, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

# on supprime ticket car ne sert à rien
X_train = X_train.drop(columns = 'Ticket')
X_test = X_test.drop(columns = 'Ticket')

# one-hot encoder pour Embarked et Sex

featoh = ['Sex', 'Embarked']

OH = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
oh_train = pd.DataFrame(OH.fit_transform(X_train[featoh]))
oh_val = pd.DataFrame(OH.transform(X_test[featoh]))

#remettre les indexes

oh_train.index = X_train.index
oh_val.index = X_test.index


X_train = X_train.drop(columns = featoh)
X_test = X_test.drop(columns = featoh)

# on recolle

X_train_f = pd.concat([X_train, oh_train], axis = 1)
X_test_f = pd.concat([X_test, oh_val], axis = 1)

X_train_f = X_train_f.drop(columns = 'Cabin')
X_test_f = X_test_f.drop(columns = 'Cabin')

test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [65]:
# création du modèle

model = XGBClassifier(random_state = 1)



In [66]:
p = {
    'n_estimators' : Integer(50, 1000, prior = 'uniform'),
    'max_depth' : Integer(3, 10, prior = 'uniform'),
    'learning_rate' : Real(0.0001, 0.3, prior = 'log-uniform'),
    'reg_alpha' : Real(0.01, 1.0, prior = 'log-uniform'),
    'reg_lambda': Real(0.001, 1.5, prior = 'log-uniform'),
}

opt = BayesSearchCV(XGBClassifier(random_state = 1), p, cv = 3, n_jobs = -1, random_state = 1, n_iter = 30)
opt.fit(X_train_f, y_train)
print(opt.best_params_)

OrderedDict({'learning_rate': 0.002606083634409021, 'max_depth': 3, 'n_estimators': 761, 'reg_alpha': 0.023277942764233707, 'reg_lambda': 0.9729063707072745})


In [ ]:
m = opt.best_estimator_
m.fit(X_train_f, y)
prds = m.predict(X_test_f)


ValueError: Found input variables with inconsistent numbers of samples: [891, 418]